# Permeation method for hydrogen transport properties in molten salt

In this notebook we'll describe the method employed to estimate the diffusivity and solubility of hydrogen in molten salt using a permeation rig.

The principle of the measurement is to have hydrogen diffusing through a membrane (here a salt, with a negligible metal membrane). The upstream pressure is kept constant and the downstream volume will experience a pressure rise. The steady state pressure rise then gives information on the salt permeability and the time-lag (detailed below) gives an insight on the diffusivity. The ratio of the permeability and diffusivity yields the solubility.

Note: having a closed downstream volume slightly differs from Fukada's experiment on FLiNaK (2006) where the authors swept the downstream gas at a constant rate.

The experimental pressure rise will be mimicked by an analytical solution and we will then quantify the error propagation.

## Generating the data
Let us first find the salt properties in the literature using HTM. We will use the Fukada 2006 paper.

In [63]:
import h_transport_materials as htm

ureg = htm.ureg
ureg.setup_matplotlib()

D_flinak = htm.diffusivities.filter(material=htm.FLINAK, author='fukada')[0]
K_flinak = htm.solubilities.filter(material=htm.FLINAK, author='fukada')[0]

print("Diffusivity:")
print(D_flinak)
print("Solubility:")
print(D_flinak)

Diffusivity:

        Author: Fukada
        Material: flinak
        Year: 2006
        Isotope: H
        Pre-exponential factor: 3.70×10⁻⁶ m²/s
        Activation energy: 4.50×10⁻¹ eV/particle
        
Solubility:

        Author: Fukada
        Material: flinak
        Year: 2006
        Isotope: H
        Pre-exponential factor: 3.70×10⁻⁶ m²/s
        Activation energy: 4.50×10⁻¹ eV/particle
        


The experimental parameters:

In [64]:
import numpy as np

T = 900 * ureg.K  # temperature
V_down = 1000 * ureg.cm**3  # downstream volume
A = 2 * ureg.cm**2  # gas/salt interface surface area 
P_up = 1 * ureg.atm  # upstream pressure
L = 40 * ureg.mm  # salt thickness

# estimate properties at operating temperature
D_flinak = D_flinak.value(T)
print(D_flinak)

K_flinak = K_flinak.value(T)

print(K_flinak)
permeability = D_flinak * K_flinak


1.1113808953304486e-08 meter ** 2 / second
2.4501912127738196e+19 particle / meter ** 3 / pascal
0.6 meter * millimeter


We can now compute the transient analytical solution describing the downstream pressure rise.

$$P_\mathrm{down}(t) = R_g \ T \frac{A}{L \ V_\mathrm{down}} \ \Phi \ P_\mathrm{up} \left[t - \frac{L^2}{6 \ D} - \frac{2 \ L^2}{\pi^2 D} \sum_{n=1}^\infty \frac{(-1)^{n+1}}{n^2} \exp{\left( -\frac{D \ n^2 \pi^2}{L^2} t \right)}\right] $$

where $P_\mathrm{up}$ is the upstream pressure, $T$ is the temperature, $V$ is the downstream volume, $L$ is the thickness of salt, $R_g$ is the gas constant, and $A$ is the surface area of the gas/salt interface in the downstream chamber.

For more details, see:
- [Review of time lag permeation technique as a method for characterisation of porous media and membranes](https://link.springer.com/article/10.1007/BF01653631)
- [Hydrogen transport and trapping in ODS-EUROFER](https://doi.org/10.1016/j.fusengdes.2007.02.002)

In [65]:
timelag = L.to(ureg.m)**2 / (6 * D_flinak)
Rg = htm.Rg * ureg.Pa * ureg.m**3 * ureg.mol**-1 * ureg.K**-1  # gas constant

# Analytical solution for transient downstream pressure rise
def downstream_pressure(t):


    sum_term = 0
    for n in np.arange(1, 1000):
        sum_term += (
            (-1) ** (n+1) / n**2 * np.exp(-D_flinak * n**2 * np.pi**2 * t / L**2)
        )

    transient_term = 2 * L**2 / (np.pi**2 * D_flinak) * sum_term
    return Rg * T / V_down * A / L * permeability * P_up * (t - timelag + transient_term)

23994.174075430572 second


In [66]:
import matplotlib.pyplot as plt

t = np.linspace(0, 10 * timelag).to(ureg.min)
p_down = downstream_pressure(t).to(ureg.Pa)

plt.plot(t, p_down)
plt.xlabel(f"Time ({t.units:~})")
plt.ylabel(f"Downstream pressure ({p_down.units:~})")
plt.show()


DimensionalityError: Cannot convert from 'meter ** 5 * minute * particle * pascal * standard_atmosphere / centimeter / kelvin / millimeter / mole ** 2 / second' ([length] * [mass] ** 2 / [substance] / [temperature] / [time] ** 4) to 'pascal' ([mass] / [length] / [time] ** 2)

We can add some noise by assuming a 5% error on the low pressure side measurement.

In [ ]:
relative_p_down_error = 5/100
relative_noise = np.random.normal(0, relative_p_down_error, t.shape)

noisy_p_down = downstream_pressure(t) * (1 + relative_noise)
noisy_p_down = noisy_p_down.to(ureg.Pa)

yerr = relative_p_down_error*noisy_p_down

plt.errorbar(t, noisy_p_down, yerr=yerr)
plt.xlabel(f"Time ({t.units:~})")
plt.ylabel(f"Downstream pressure ({noisy_p_down.units:~})")
plt.show()

We fit the steady state pressure rise for $ t > 1200 \ \mathrm{min} $ with a linear function.


In [ ]:
from scipy.optimize import curve_fit

# select only t > 1200 min
t_min_fit = 1200 * ureg.min
indices_fit = np.where(t > t_min_fit)
t_data_fit = t[indices_fit]
p_down_data_fit = noisy_p_down[indices_fit]
data_sigma_fit = relative_p_down_error * noisy_p_down[indices_fit]

# fit the curve

def linearFunc(x, intercept, slope):
    y = intercept + slope * x
    return y

res, cov = curve_fit(
    linearFunc,
    t_data_fit.to(ureg.s).magnitude,
    p_down_data_fit.to(ureg.Pa).magnitude,
    sigma=data_sigma_fit.to(ureg.Pa).magnitude,
    absolute_sigma=True,
)

fitted_intercept = res[0] * ureg.Pa
fitted_slope = res[1] * ureg.Pa * ureg.s**-1

# compute the uncertainties of slope and intercept
d_inter = np.sqrt(cov[0][0]) * ureg.Pa
d_slope = np.sqrt(cov[1][1]) * ureg.Pa * ureg.s**-1


print(f"Slope: {fitted_slope:.2e~P} with uncertainty {d_slope:.2e~P}")
print(f"Intercept: {fitted_intercept:.2f~P} with uncertainty {d_inter:.2f~P}")


In [ ]:
timelag = -fitted_intercept/fitted_slope
t_fit = np.linspace(timelag, t.max()).to(ureg.min)

plt.errorbar(t, noisy_p_down, yerr=relative_p_down_error*noisy_p_down)
plt.plot(t_fit, linearFunc(t_fit, fitted_intercept, fitted_slope))

plt.axvline(x=timelag, linestyle="dashed", color="tab:grey")
plt.annotate(r"$\tau_\mathrm{timelag}$", (timelag*1.1, 300*ureg.Pa))

plt.xlabel(f"Time ({t.units:~})")
plt.ylabel(f"Downstream pressure ({noisy_p_down.units:~})")
plt.ylim(bottom=0)
plt.xlim(left=0)
plt.show()

## Determination of H properties and uncertainty quantification

We use the `uncertainties` package to compute the error propagation of the permeation method.

The uncertainties associated with the slope and intercept are computed by `curve_fit` (see above).

Temperature: $\Delta T = 0.75 \ \%$

Upstream pressure: $\Delta P = 0.5 \ \%$

Downstream pressure: $\Delta P = 5 \ \%$ (already included in the noise, see above)

In [ ]:
from uncertainties import ufloat

slope_u = ufloat(
    fitted_slope.to(ureg.Pa * ureg.s**-1).magnitude,
    d_slope.to(ureg.Pa * ureg.s**-1).magnitude,
)
slope_u *= ureg.Pa * ureg.s**-1


intercept_u = ufloat(
    fitted_intercept.to(ureg.Pa).magnitude,
    d_inter.to(ureg.Pa).magnitude,
)
intercept_u *= ureg.Pa

T = T.to(ureg.K)
T_u = ufloat(T.magnitude, 0.75 / 100 * T.magnitude) * ureg.K

P_up = P_up.to(ureg.Pa)
P_up_u = ufloat(P_up.magnitude, 0.5 / 100 * P_up.magnitude) * ureg.Pa


### Permeability

The permeability $\Phi$ is given by:

$$\Phi = \frac{S}{P_\mathrm{up} \ T} \ \frac{V \ L}{R_g \ A} $$

where $S$ is the slope of the pressure rise.

In [ ]:
permeability_u = slope_u * P_up_u**-1 * T_u**-1 * V_down * L / Rg / A
permeability_u = permeability_u.to(ureg.particle * ureg.m**-1 * ureg.Pa**-1 * ureg.s**-1)

print(f"Measured: {permeability_u:~P}")
print(f"Real: {D_flinak*K_flinak:.2e~P}")

### Diffusivity

The timelag $\tau_\mathrm{timelag}$ is defined as:
$$\tau_\mathrm{timelag} = \frac{L^2}{6 \ D}$$

where $D$ is the diffusivity.


The timelag corresponds to the time when the linear pressure rise curve intersects with the X axis.

$$P(t=\tau_\mathrm{timelag}) = 0$$

wich can be written as:

$$S \ \tau_\mathrm{timelag} + I = 0$$

where $S$ and $I$ are respectively the slope and the intercept of the steady pressure rise curve.


The diffusivity $D$ can therefore be expressed by:

$$D = \frac{S}{6 \ I} \ L^2$$


In [ ]:
diffusivity_u = -slope_u/(6 * intercept_u) * L**2
diffusivity_u = diffusivity_u.to(ureg.m**2 * ureg.s**-1)

print(f"Measured: {diffusivity_u:~P}")
print(f"Real: {D_flinak:.2e~P}")

### Solubility
The solubility $K_H$ is given as:

$$K_H = \Phi / D$$

In [ ]:
solubility_u = permeability_u / diffusivity_u
solubility_u = solubility_u.to(ureg.particle * ureg.m**-3 * ureg.Pa**-1)

print(f"Measured: {solubility_u:~P}")
print(f"Real: {K_flinak:.2e~P}")

## Conclusion

The permeation method is a well-established method for measuring diffusivities and solubilities of metals but also molten salts.

The uncertainty associated with diffusivities and solubilities are reasonable ($<10 \ \%$).
Moreover, efforts aiming to reduce the uncertainty of the downstream pressure measurement will help driving the overall error down.

### TODO: what would be the error associated with geometrical measurements?